# Custom Grid-World

In [1]:
import sys
sys.path += ['../build', '../../../build', '../../lib']
print(sys.version)
import numpy as np
import libry as ry

3.8.10 (default, Jun 22 2022, 20:18:18) 
[GCC 9.4.0]
**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6

## Setting up a basic Config

In [2]:
C = ry.Config()
C.view()

Add base-grid with walls

In [3]:
C.clear()
C.addFile('../rai-robotModels/test/base-grid.g')
C.addFile('../rai-robotModels/pr2/pr2.g');

## Generating World

Set some variables.

In [4]:
colors = [[.8, .1, .1], [1., .3, .2]]
world = np.array([
    [1, 1, 1, 1, 1, 1, 1],
    [1, 4, 0, 0, 0, 0, 1],
    [1, 0, 3, 1, 3, 0, 1],
    [1, 1, 0, 1, 0, 1, 1],
    [1, 0, 2, 0, 3, 0, 1],
    [1, 4, 0, 0, 0, 4, 1],
    [1, 1, 1, 1, 1, 1, 1]
])

off = -3

Iterate over world Array and generate Environment.

In [5]:
for x in range(world.shape[0]):
    for y in range(world.shape[1]):
        if(world[x][y] == 1):
            c = colors[(x+y) % 2]
            C.addObject(name='block', shape=ry.ST.ssBox, pos=[x+off, y+off, .5], size=[1, 1, 1, .02], color=c)
        
        if(world[x][y] == 2):
            C.selectJointsByTag(["base"])
            C.setJointState([x+off, y+off, 0])
        
        if(world[x][y] == 3):
            box = C.addObject(name='box', shape=ry.ST.ssBox, pos=[x+off, y+off, .45], size=[.9, .9, .9, .02], color=[.6, .3, 0.])
        
        if(world[x][y] == 4):
            C.addObject(name='goal', shape=ry.ST.ssBox, pos=[x+off, y+off, .05], size=[.95, .95, .1, .05], color=[0., 1., 0.])


## Collision
Calculate Collision between All Frames.

In [129]:
coll = C.feature(ry.FS.accumulatedCollisions, [])

C.computeCollisions() #collisions/proxies are not automatically computed on set...State
coll.eval(C)

# C.getCollisions(0)

(array([0.]), array([[0., 0., 0.]]))

Calculate Collision (or other Features) between two Frames.

In [130]:
dist = C.feature(ry.FS.distance, ['coll_wrist_r', 'box'])
dist.eval(C)

(array([-1.73706171]), array([[-0.99909709,  0.04248524, -0.49631358]]))

## Path Optimization

In [8]:
X0 = C.getFrameState()

obj1 = "box";
arm = "pr2L";

komo = C.komo_path(1., 20, 10., True)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
# komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
# komo.addObjective([1.], ry.FS.distance, [arm, obj1], ry.OT.eq, [1e2]);

komo.addObjective(times=[1.], feature=ry.FS.position, frames=['base'], type=ry.OT.eq, scale=[1, 1, 0], target=[1, 1, 0]);


komo.optimize()
komo.getReport()
komo.view(False, "my solution")

** KOMO::run solver:sparse collisions:1 x-dim:60 T:20 k:2 phases:1 stepsPerPhase:20 tau:0.5  #timeSlices:22 #totalDOFs:60 #frames:4378
** optimization time:0.202076 (kin:0.02007 coll:0.127793 feat:0.043001 newton: 0.004083) setJointStateCount:46
   sos:0.00277356 ineq:0 eq:2


0

In [10]:
komo.view_play(False, .1)

0